In [10]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2
import os
import glob

import functions as fn

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

seed=42

In [3]:
train_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/train', 
    labels='inferred',
    subset="training",
    validation_split=.2,
    seed=seed,
    shuffle=False)

val_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/train', 
    labels='inferred',
    subset="validation",
    validation_split=.2,
    seed=seed,
    shuffle=False)

test_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/test', 
    labels='inferred',
    shuffle=False)

Found 317 files belonging to 2 classes.
Using 254 files for training.
Found 317 files belonging to 2 classes.
Using 63 files for validation.
Found 78 files belonging to 2 classes.


In [4]:

for images, labels in train_data.take(1):
    for i in range(3):
        fn.display_image(images[i].numpy().astype('uint8'))

In [5]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [6]:
train_data = val_data.map(preprocess).prefetch(1)
val_data = val_data.map(preprocess).prefetch(1)
test_data = test_data.map(preprocess).prefetch(1)

In [11]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)

for layer in base_model.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(1, activation = 'sigmoid')(avg)
model = keras.Model(inputs = base_model.input, outputs = output)

optimizer = keras.optimizers.SGD(lr = 0.2, momentum = 0.9, decay = 0.01)
model.compile(loss = 'binary_crossentropy', optimizer = optimizer,  metrics = ['accuracy', recall_m, precision_m, f1_m])

results = model.fit(train_data, epochs = 5, validation_data = val_data)

Epoch 1/5


c:\Users\nicho\anaconda3\envs\capstone-env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2/2 [==============================] - 5s 2s/step - loss: 0.2731 - accuracy: 0.9683 - recall_m: 0.9688 - precision_m: 1.0000 - f1_m: 0.9839 - val_loss: 4.9581e-06 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_m: 1.0000
Epoch 2/5
2/2 [==============================] - 2s 2s/step - loss: 2.2797e-06 - accuracy: 1.0000 - recall_m: 1.0000 - precision_m: 1.0000 - f1_m: 1.0000 - val_loss: 2.4778e-09 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_m: 1.0000
Epoch 3/5
2/2 [==============================] - 2s 2s/step - loss: 5.7846e-10 - accuracy: 1.0000 - recall_m: 1.0000 - precision_m: 1.0000 - f1_m: 1.0000 - val_loss: 7.8538e-12 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_m: 1.0000
Epoch 4/5
2/2 [==============================] - 2s 2s/step - loss: 1.2720e-12 - accuracy: 1.0000 - recall_m: 1.0000 - precision_m: 1.0000 - f1_m: 1.0000 - val_loss: 8.0246e-14 - val_accuracy: 1.0000 - val_reca

In [13]:
model.evaluate(test_data)

3/3 [==============================] - 1s 400ms/step - loss: 26.3221 - accuracy: 0.4872 - recall_m: 0.6667 - precision_m: 0.5833 - f1_m: 0.6190        


[26.322065353393555,
 0.4871794879436493,
 0.6666666865348816,
 0.5833333134651184,
 0.6190475821495056]

In [ ]:
acc = results.history['accuracy']
val_acc = results.history['val_accuracy']

loss = results.history['loss']
val_loss = results.history['val_loss']

recall = results.history['recall_m']
val_recall = results.history['val_recall_m']

precision = results.history['precision_m']
val_precision = results.history['val_precision_m']

f1 = results.history['f1_m']
val_f1 = results.history['f1_m']

epochs_range = range(len(results.history['accuracy']))

plt.figure(figsize=(22, 18))
plt.subplot(2, 3, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 3, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.subplot(2, 3, 3)
plt.plot(epochs_range, recall, label='Training Recall')
plt.plot(epochs_range, val_recall, label='Validation Recall')
plt.legend(loc='lower right')
plt.title('Training and Validation Recall')
plt.show()

plt.subplot(2, 3, 4)
plt.plot(epochs_range, precision, label='Training Precision')
plt.plot(epochs_range, val_precision, label='Validation Precision')
plt.legend(loc='lower right')
plt.title('Training and Validation Precision')
plt.show()

plt.subplot(2, 3, 5)
plt.plot(epochs_range, f1, label='Training F1')
plt.plot(epochs_range, val_f1, label='Validation F1')
plt.legend(loc='lower right')
plt.title('Training and Validation F1')
plt.show()

: 

: 